# TODO

- We are statically passing 4 indexes each time, this should be dynamic
- It should dynamically handle NaN values

## Load returns data from CSV

In [3]:
import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Load in CSV data
df = pd.read_csv('regreturns.csv')

df.describe()


,MSCI AC Asia Pacific GR CAD,MSCI EM GR CAD,MSCI Europe GR CAD,MSCI North America GR CAD,MSCI World Growth GR CAD,MSCI World Large GR USD,MSCI World Small Cap GR CAD,MSCI World Value GR CAD,TAF Generation,Beutel Goodman Global Equity,Burgundy Global Equity,Manulife Trimark Fund
count,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,72.000000,122.000000,121.000000,122.000000
mean,0.692295,0.659754,0.562787,0.976393,0.901066,0.792377,0.995574,0.702623,1.795694,0.791475,1.014959,0.828689
std,3.677336,4.724160,4.371667,3.231191,3.414757,3.305916,3.917727,3.463996,2.978335,3.457786,3.055088,3.554607
min,-13.780000,-17.230000,-14.800000,-9.320000,-13.410000,-11.120000,-12.980000,-10.850000,-4.860000,-9.630000,-8.160000,-10.880000
25%,-0.975000,-2.012500,-1.747500,-0.585000,-0.862500,-0.820000,-1.112500,-1.182500,-0.002500,-1.010000,-0.420000,-0.512500
50%,0.670000,0.695000,0.820000,1.105000,1.290000,0.930000,1.550000,1.165000,1.665000,1.225000,1.010000,1.005000
75%,2.787500,3.560000,3.645000,3.300000,3.127500,3.052500,3.680000,2.855000,3.570000,3.180000,3.080000,3.147500
max,11.470000,13.420000,9.480000,7.720000,8.640000,7.330000,10.100000,7.710000,9.830000,8.790000,9.280000,8.350000


## Separate data into arrays

Data is separated into different arrays.  One-dimensional arrays for all Funds, multi-dimensional arrays for the indexes we are testing against.

In [14]:
xGeo = df.iloc[:,1:5].values
xStyle = df.iloc[:,5:9].values
yGeneration = df.iloc[:,9].values
yBG = df.iloc[:,10].values
yBurgundy = df.iloc[:,11].values
yTrimark = df.iloc[:,12].values
xCapSize = df.iloc[:,6:8].values
xStyleOnly = df[['MSCI World Growth GR CAD', 'MSCI World Value GR CAD']].values

## Optimization Problem

Minimize $$\epsilon_t = R_t - (w_1*I_{1,t} + w_2*I_{2,t} + ... + w_n*I_{n,t})$$

Subject to $$w_i \ge 0$$ $$\sum_{i=1}^n w_i = 1$$

### Set up optimization functions

In [5]:
def objective(B,y,X):
    u = np.subtract(y,np.dot(X,B))
    return np.dot(u.T,u)

def constraint(B):
    return np.sum(B) - 1.0

bounds = (0.0,1.0)

In [28]:
from scipy.optimize import minimize

# Initialize Beta weights of zero
B = np.zeros(4)

# Set constraint properties
const = {'type':'eq','fun':constraint}

# Set bounds for each weight (4 in this case)
bnds = (bounds,bounds,bounds,bounds)

In [29]:
# Set up scipy solution
sol = minimize(objective,B,args=(yBG,xStyle),method='SLSQP',bounds=bnds,constraints=const)
sol

     fun: 108.43551538677978
     jac: array([ 54.10956097,  52.2811079 ,  52.28291988,  52.28112507])
 message: 'Optimization terminated successfully.'
    nfev: 50
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([  1.42247325e-16,   4.69910512e-01,   1.37426503e-01,
         3.92662985e-01])

In [30]:
df.iloc[:,5:9].columns

Index(['MSCI World Growth GR CAD', 'MSCI World Large GR USD',
       'MSCI World Small Cap GR CAD', 'MSCI World Value GR CAD'],
      dtype='object')

### Run style and cap size separately and compare results

In [26]:
# Initialize Beta weights of zero
B = np.zeros(2)

# Set constraint properties
const = {'type':'eq','fun':constraint}

# Set bounds for each weight (4 in this case)
bnds = (bounds,bounds)

solStyle = minimize(objective,B,args=(yBG,xStyleOnly),method='SLSQP',bounds=bnds,constraints=const)

print(df[['MSCI World Growth GR CAD', 'MSCI World Value GR CAD']].columns)
solStyle

Index(['MSCI World Growth GR CAD', 'MSCI World Value GR CAD'], dtype='object')


     fun: 111.54384267188676
     jac: array([ 35.06569862,  35.06569767])
 message: 'Optimization terminated successfully.'
    nfev: 13
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([ 0.29971937,  0.70028063])

In [27]:
solCap = minimize(objective,B,args=(yBG,xCapSize),method='SLSQP',bounds=bnds,constraints=const)

print(df.iloc[:,6:8].columns)
solCap

Index(['MSCI World Large GR USD', 'MSCI World Small Cap GR CAD'], dtype='object')


     fun: 119.9421641707696
     jac: array([ 43.64182854,  43.64182758])
 message: 'Optimization terminated successfully.'
    nfev: 16
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 0.83777647,  0.16222353])

## Results

When running the regression with all 4 indexes, the ratios are: <br>
Value:Growth >>> Infinite <br>
Large:Small >>> 10:3 <br> <br>

When running the cap size and style indexes in separate regressions, the ratios are: <br>
Value:Growth >>> 7:3 <br>
Large:Small >>> 5:1